<a href="https://colab.research.google.com/github/ingabLee/low-code-book-projects/blob/main/keras_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

!wget -q https://storage.googleapis.com/low-code-ai-book/car_prices_train.csv
!wget -q https://storage.googleapis.com/low-code-ai-book/car_prices_valid.csv
!wget -q https://storage.googleapis.com/low-code-ai-book/car_prices_test.csv

train_df = pd.read_csv('./car_prices_train.csv')
y_train = train_df['sellingprice']
X_train = train_df.drop('sellingprice', axis=1)

valid_df = pd.read_csv('./car_prices_valid.csv')
y_valid = valid_df['sellingprice']
X_valid = valid_df.drop('sellingprice', axis=1)

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import( StringLookup, HashedCrossing,
                                    Discretization, Concatenate)
cat_cols = ['make', 'model', 'trim', 'body', 'transmission', 'state',
            'color', 'interior']

num_cols = ['odometer', 'year', 'condition']

inputs = {}

for col in cat_cols:
  inputs[col] = tf.keras.Input(shape=(1,), name=col, dtype=tf.string)

for col in num_cols:
  inputs[col] = tf.keras.Input(shape=(1,), name=col, dtype=tf.int64)

In [9]:
preproc_layers = {}
for col in cat_cols:
  layer = StringLookup(output_mode='one_hot')
  layer.adapt(X_train[col])
  preproc_layers[col] = layer(inputs[col])

for col in num_cols:
  layer = Discretization(num_bins=10, output_mode='one_hot')
  layer.adapt(X_train[col])
  preproc_layers[col] = layer(inputs[col])

In [11]:
model_trim = tf.keras.layers.HashedCrossing(num_bins=1000, output_mode='one_hot')(
    (inputs['model'], inputs['trim']))

color_int = tf.keras.layers.HashedCrossing(num_bins=400, output_mode='one_hot')(
    (inputs['color'], inputs['interior']))

preproc_layers['model_trim'] = model_trim
preproc_layers['color_int'] = color_int

In [12]:
prepared_layer = Concatenate()(preproc_layers.values())
prepared_layer = tf.reshape(prepared_layer, [-1, 3903])

hid_1 = tf.keras.layers.Dense(16, activation='relu')(prepared_layer)
hid_2 = tf.keras.layers.Dense(16, activation='relu')(hid_1)
output = tf.keras.layers.Dense(1)(hid_2)

model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='mse')

train_ds = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train)).batch(100)
valid_ds = tf.data.Dataset.from_tensor_slices((dict(X_valid), y_valid)).batch(1000)

history = model.fit(
    x=train_ds, epochs=25, verbose=1, validation_data=valid_ds)

ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: dict_values([<KerasTensor shape=(None, 54), dtype=float32, sparse=False, name=keras_tensor_9>, <KerasTensor shape=(None, 766), dtype=float32, sparse=False, name=keras_tensor_10>, <KerasTensor shape=(None, 1490), dtype=float32, sparse=False, name=keras_tensor_11>, <KerasTensor shape=(None, 86), dtype=float32, sparse=False, name=keras_tensor_12>, <KerasTensor shape=(None, 3), dtype=float32, sparse=False, name=keras_tensor_13>, <KerasTensor shape=(None, 35), dtype=float32, sparse=False, name=keras_tensor_14>, <KerasTensor shape=(None, 21), dtype=float32, sparse=False, name=keras_tensor_15>, <KerasTensor shape=(None, 18), dtype=float32, sparse=False, name=keras_tensor_16>, <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_17>, <KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_18>, <KerasTensor shape=

In [13]:
!pip install -q keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt
from functools import partial

def _build_model_fn(hp, prepared_layer):
  units_1 = hp.Int('units_1', min_value=8, max_value=64, step=4)
  units_2 = hp.Int('units_2', min_value=4, max_value=64, step=4)
  units_3 = hp.Int('')